# Control Systems 1, Coding Exercise 02: Jupyter Notebook on Introduction to System Classification and Properties
2024 ETH Zurich, Mark Benazet Castells, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli


This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers the basics of system modeling, classification, and analysis, with a focus on building intuition through visual and interactive examples.

Authors:
- Mark Benazet Castells; mbenazet@ethz.ch
- Felix Muller; fmuller@ethz.ch

## Learning Objectives

After completing this notebook, you should be able to:

1. Understand input-output models of linear time-invariant (LTI) systems.
2. Write the model of an LTI system with A, B, C, D matrices.
3. Understand the concept of "state" to model a system.
4. Classify systems as static / dynamic, causal/non-causal, time-invariant/time-varying, linear/nonlinear.
5. Understand the notion of equilibrium points.
6. Approximate a non-linear system with a linear one.

# Setup

## Installing the required packages:
The following packages are required to run this notebook. If you haven't installed them yet, you can do so by running the following cell.

In [ ]:
!pip install numpy matplotlib scipy ipywidgets

## Import the packages:
The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact, interactive, fixed, widgets, FloatSlider, Dropdown
from IPython.display import display, Markdown, Latex, Image

## 1. Input-Output Models

An input-output model is a map Σ from an input signal u(t) to an output signal y(t):

$$y = \Sigma u$$

or more explicitly:

$$y(t) = (\Sigma u)(t), \quad \forall t \in T$$

This can be represented by a block diagram:

<div style="text-align:center;">
    <img src="./img/block_dgm.png" alt="Block Diagram" width="500">
</div>

The easiest example of an input-output mode would be $y = u$, This is a simple linear relationship between the input and output signals.

In [ ]:
def linear_function(k):
    u = np.linspace(-10, 10, 200)
    y = k * u
    
    plt.figure(figsize=(10, 6))
    plt.plot(u, y)
    plt.title(f'Linear Function: y=u')
    plt.xlabel('Input (u)')
    plt.ylabel('Output (y)')
    plt.grid(True)
    plt.axhline(y=0, color='k', linestyle='--')
    plt.axvline(x=0, color='k', linestyle='--')
    plt.show()

display(linear_function(1))

Depending on how Σ affects u, we classify the system in different ways:
- Linear vs. Nonlinear
- Causal vs. Non-causal
- Static (memoryless) vs. Dynamic
- Time-invariant vs. Time-varying

## 1. Linear vs. Nonlinear Systems

A system is linear if it satisfies two properties:
- Additivity: $\Sigma(u_1 + u_2) = \Sigma u_1 + \Sigma u_2$
- Homogeneity: $\Sigma(ku) = k\Sigma u$

For any input signals $u_1$ and $u_2$, and any scalar $k$.

# Checking System Linearity

A system is linear if it satisfies the superposition principle:

$\Sigma(a u_1 + b u_2) = a\Sigma(u_1) + b\Sigma(u_2)$

Or in other words:

$y(a u_1 + b u_2) = a y(u_1) + b y(u_2)$


Where Σ represents the system, u₁ and u₂ are input signals, and a and b are scalars.

Use the interactive plot below to check this property for different systems:

- Linear system: $y = u$
- Nonlinear system: $y = u^2$
- Nonlinear system: $y = \sin(u)$

Parameters:
- $a, b$: Scalars for the superposition principle

Adjust the parameters and observe:
- If the solid and dashed lines overlap for all inputs, the system is linear.
- If they diverge, the system is nonlinear.

In [ ]:
def system_response(system_type, u):
    if system_type == 'Linear':
        return u
    elif system_type == 'Nonlinear (Quadratic)':
        return u**2
    elif system_type == 'Nonlinear (Sine)':
        return np.sin(u)

def check_linearity(system_type, a, b):
    u1 = np.linspace(-2, 2, 200)
    u2 = np.linspace(-2, 2, 200)
    
    # Individual responses
    y1 = system_response(system_type, u1)
    y2 = system_response(system_type, u2)
    
    # Additivity check
    y_sum = system_response(system_type, a*u1 + b*u2)
    y_individual_sum = a*y1 + b*y2
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Additivity plot
    ax.plot(u1, y_sum, label='Σ(au₁ + bu₂)')
    ax.plot(u1, y_individual_sum, label='aΣ(u₁) + bΣ(u₂)', linestyle='--')
    ax.set_title('Linearity Check')
    ax.set_xlabel('Input')
    ax.set_ylabel('Output')
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Create interactive widgets
system_dropdown = Dropdown(options=['Linear', 'Nonlinear (Quadratic)', 'Nonlinear (Sine)'], 
                           value='Linear', description='System:')
a_slider = FloatSlider(min=-2, max=2, step=0.1, value=1, description='a:')
b_slider = FloatSlider(min=-2, max=2, step=0.1, value=1, description='b:')

# Create interactive plot
interactive_plot = interactive(check_linearity, system_type=system_dropdown, a=a_slider, b=b_slider)

# Display interactive plot
display(interactive_plot)